# Results

The following code generates choropleth maps showing the number of cyclable trips in São Paulo districts.

In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2
import saopaulo.sp_grid as gr
import bikescience.distributions as dist
import saopaulo.cycling_potential as cp
import saopaulo.choropleth_folium as choro_folium
import saopaulo.choropleth as choro
from bikescience.slope import plot_slope, plot_slopes, split_route
from shapely.geometry import LineString, Point
import ast
from numpy import nan
import math
from bikescience.arrow import draw_arrow

import folium
import pandas as pd
import geopandas as gpd
from ipywidgets import interact_manual, widgets, fixed
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import warnings
import numpy as np
warnings.simplefilter('ignore')

gr.SP_LAT = -23.63
gr.SP_LON = -46.55

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/home/pedro/.local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
districts_name = [
    'Bela Vista', 'Bom Retiro', 'Cambuci', 'Consolação', 'Liberdade', 
    'República', 'Santa Cecília', 'Sé','Butantã', 'Morumbi', 'Raposo Tavares', 
    'Rio Pequeno', 'Vila Sônia', 'Barra Funda', 'Jaguara', 'Jaguaré', 'Lapa', 
    'Perdizes', 'Vila Leopoldina', 'Alto de Pinheiros', 'Itaim Bibi', 
    'Jardim Paulista', 'Pinheiros','Aricanduva', 'Carrão', 'Vila Formosa',
    'Água Rasa', 'Belém', 'Brás', 'Mooca', 'Pari', 'Tatuapé', 'Artur Alvim',
    'Cangaíba', 'Penha', 'Vila Matilde', 'São Lucas', 'Sapopemba',
    'Vila Prudente','Cidade Tiradentes', 'Ermelino Matarazzo', 'Ponte Rasa',
    'Guaianases', 'Lajeado', 'Itaim Paulista', 'Vila Curuçá', 'Cidade Líder',
    'Itaquera', 'José Bonifácio', 'Parque do Carmo', 'Iguatemi', 'São Mateus',
    'São Rafael', 'Jardim Helena', 'São Miguel', 'Vila Jacuí','Cachoeirinha', 
    'Casa Verde', 'Limão', 'Brasilândia', 'Freguesia do Ó', 'Anhanguera', 
    'Perus', 'Jaraguá', 'Pirituba', 'São Domingos','Jaçanã', 'Tremembé', 
    'Mandaqui', 'Santana', 'Tucuruvi', 'Vila Guilherme', 'Vila Maria', 
    'Vila Medeiros','Cursino', 'Ipiranga', 'Sacomã', 'Jabaquara', 'Moema', 
    'Saúde', 'Vila Mariana', 'Campo Limpo', 'Capão Redondo', 'Vila Andrade', 
    'Cidade Dutra', 'Grajaú', 'Socorro', 'Cidade Ademar', 'Pedreira', 
    'Jardim Ângela', 'Jardim São Luís', 'Marsilac', 'Parelheiros',
    'Campo Belo', 'Campo Grande', 'Santo Amaro']

In [3]:
#read data

districts = True
OD_zone = !districts 

if districts:
    zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Distritos_2017_region.shp')
    zone_shp.crs = {'init': 'epsg:31983'}  
    zone_shp.to_crs(epsg='4326', inplace=True)
    # converting to km^2
    zone_shp['Area_ha_2'] = zone_shp['Area_ha'] / 100
    zone_shp['NumeroZona'] = zone_shp['NumeroDist']
    zone_shp['NomeZona'] = zone_shp['NomeDistri']
    zone_shp['NumeroMuni'] = 0

    numeroMuni = []
    for _, d in zone_shp.iterrows():
        if d['NomeDistri'] in districts_name:
            numeroMuni.append(36)
        else:
            numeroMuni.append(0)
    zone_shp['NumeroMuni'] = numeroMuni
    
else:
    zone_shp = gpd.read_file('../data/sao-paulo/od/shapes/Zonas_2017_region.shp')
    zone_shp.crs = {'init': 'epsg:31983'}  
    zone_shp.to_crs(epsg='4326', inplace=True)

the_grid = gr.create(n=9, west_offset=-0.15, east_offset=0.23, north_offset=0.19, south_offset=-0.46)

In [4]:
od_trips = gpd.read_file('bases/routes_potential.shp')

In [7]:
display(od_trips.head(3))

,ID_ORDEM,ID_ORDEM.1,ID_ORDEM_1,orig_lat,orig_lon,dest_lat,dest_lon,ZONA,MUNI_DOM,CO_DOM_X,...,ID_ORDEM_2,length,modal,id,distance_p,age_potent,inclinatio,final_pote,final_po_1,geometry
0,1.0,1.0,1.0,-23.551678,-46.628858,-23.551495,-46.635115,1.0,36.0,333743.0,...,1.0,1849.286293,pedestrian,1.0,0.650491,0.023107,0.206501,0.293366,0.428496,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."
1,2.0,2.0,2.0,-23.551495,-46.635115,-23.551678,-46.628858,1.0,36.0,333743.0,...,2.0,1760.427711,pedestrian,2.0,0.689523,0.023107,0.463706,0.392112,0.576615,"LINESTRING Z (-46.63515 -23.55155 783.14000, -..."
2,3.0,3.0,3.0,-23.551678,-46.628858,-23.571829,-46.690238,1.0,36.0,333743.0,...,3.0,9473.535456,subway,3.0,0.032389,0.848482,0.066909,0.315927,0.049649,"LINESTRING Z (-46.62883 -23.55186 748.52000, -..."


In [10]:

print('------------------------------')
print('--------- ALL TRIPS ----------')
print('------------------------------')
print('')

print('Trips:                             ', 
      round(sum(od_trips['FE_VIA'])))

print('Trips (filtering age):             ', 
      round(sum(od_trips
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]['FE_VIA'])))

print('Cyclable Trips:                     ', 
      round(sum(od_trips.loc[od_trips['final_po_1'] > .75]['FE_VIA'])))

print('Cyclable Trips (filtering age):     ', 
      round(sum(od_trips
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]
                .loc[od_trips['final_po_1'] > .75]['FE_VIA'])))


#print('Total de viagens cicláveis    (> 0.7):', 
#      round(sum(od_trips.loc[od_trips['final_potential'] > .7]['FE_VIA'])))

print('')
print('')
print('------------------------------')
print('----------- CAR --------------')
print('------------------------------')
print('')
print('Trips:                              ', 
      round(sum(od_trips
                .loc[od_trips['modal'] == 'car']['FE_VIA'])))

print('Trips (filtering age):               ', 
      round(sum(od_trips 
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]
                .loc[od_trips['modal'] == 'car']['FE_VIA'])))


#print('Total de viagens cicláveis    (carro) (> 0.7):', 
#      round(sum(od_trips.loc[od_trips['modal'] == 'car'].loc[od_trips['final_potential'] > .7]['FE_VIA'])))

print('Cyclable Trips:                      ', 
      round(sum(od_trips
                .loc[od_trips['modal'] == 'car']
                .loc[od_trips['final_po_1'] > .75]['FE_VIA'])))

print('Cyclable Trips (filtering age):      ', 
      round(sum(od_trips
                .loc[od_trips['IDADE'] > 16]
                .loc[od_trips['IDADE'] < 65]
                .loc[od_trips['modal'] == 'car']
                .loc[od_trips['final_po_1'] > .75]['FE_VIA'])))

------------------------------
--------- ALL TRIPS ----------
------------------------------

Trips:                              49273751
Trips (filtering age):              37235661
Cyclable Trips:                      7173186
Cyclable Trips (filtering age):      4341846


------------------------------
----------- CAR --------------
------------------------------

Trips:                               11583313
Trips (filtering age):                9150258
Cyclable Trips:                       1671930
Cyclable Trips (filtering age):       1152359


In [11]:
# calculate the number of trips that passes through 
def calculate_trips_zone_intersection (trips, zones, reference):
    
    trips_geometries = list(trips['geometry'])
    trips_expansion = list(trips['FE_VIA'])

    trips_per_zone = [0]*len(zone_shp) # hash to store trips indexed by zones

    progress = 0
    
    for z in range(len(zones)):
        zone = zones.iloc[z]['geometry']
        
        if z / len(zones) >= progress:
            print (round(progress * 100), '%')
            progress += 0.25
            
        for i in range (len (trips_geometries)):
            if reference == 'intersection':
                if (trips_geometries[i].intersects(zone)):
                    trips_per_zone[z] += trips_expansion[i]
            else:
                p = trips_geometries[i].coords[reference]
                if (Point(p[0], p[1]).within(zone)):
                    trips_per_zone[z] += trips_expansion[i]
    return trips_per_zone   

In [12]:
def filter_potential_widgets(btn_text, process_function):
    
    width = '200px'
    layout = widgets.Layout(width = '500px')
    
    im = interact_manual(
        process_function,
        final_potential = widgets.FloatRangeSlider(
            value=[0.8, +1.],
            min=0, max=1., step=0.05,
            description='Cycling Potential:\t',
            style={'description_width': width},
            layout=layout
        ),
        distance_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Potential (distance):\t',
            style={'description_width': width},
            layout=layout
        ),
        inclination_potential = widgets.FloatRangeSlider(
            value=[0, +1.],
            min=0, max=1., step=0.05,
            description='Potential (slope):\t',
            style={'description_width': width},
            layout=layout
        ),
       # age_potential = widgets.FloatRangeSlider(
       #     value=[0, +1.],
       #     min=0, max=1., step=0.05,
       #     description='Age Potential:\t',
       #     style={'description_width': width},
       #     layout=layout
       # ),
        distance = widgets.FloatRangeSlider(
            value=[2, 50],
            min=0, max=50, step=0.05,
            description='Distance (km):\t',
            style={'description_width': width},
            layout=layout
        ),
        
        modal = widgets.Dropdown(options = [('All', None), ('Car', 'car'), ('Subway', 'subway'),
                                            ('Pedestrian', 'pedestrian'), ('Motorcycle', 'motorcycle'),
                                            ('Train', 'train')],
                                 description = 'Transport Modal', 
                                 style={'description_width': width}),
        reference = widgets.Dropdown(options = [('Whole Trip', 'intersection'),('Origin', 0), 
                                                ('Destination', -1)],
                                     description = 'Trip Reference:\t', 
                                     style={'description_width': width}),
        
        density = widgets.Checkbox(description = 'Density'),
        debug = widgets.Checkbox(description = 'Show trips'),
        plot_rmsp = widgets.Checkbox(description = 'Metropolitan Area'),
        title = widgets.Text(description = 'Map Title', style={'description_width': width}),
    )
    im.widget.children[10].description = 'Generate Map'
    
def inside_limits (limits, value):
    return value >= limits[0] and value <= limits[1]
    
def filter_potential(final_potential, distance_potential, 
                     #age_potential, 
                     inclination_potential, distance, modal):
    if modal != None:
        print(modal)
        aux = od_trips.loc[od_trips['modal'] == modal]
    else:
        aux = od_trips
    
    #aux = aux.loc[aux['age_potential'] >= age_potential[0]]
    #aux = aux.loc[aux['age_potential'] <= age_potential[1]]
    aux = aux.loc[aux['distance_p'] >= distance_potential[0]]
    aux = aux.loc[aux['distance_p'] <= distance_potential[1]]
    aux = aux.loc[aux['inclinatio'] >= inclination_potential[0]]
    aux = aux.loc[aux['inclinatio'] <= inclination_potential[1]]
    aux = aux.loc[aux['final_po_1'] >= final_potential[0]]
    aux = aux.loc[aux['final_po_1'] <= final_potential[1]]
    aux = aux.loc[aux['length'] >= distance[0] * 1000]
    aux = aux.loc[aux['length'] <= distance[1] * 1000]
    return aux

In [13]:
def plot_potential_trips_per_zone(final_potential, distance_potential, inclination_potential, 
                                  #age_potential, 
                                  distance,
                                  modal, reference, 
                                  debug=False, plot_rmsp = False, density = False,
                                  title = None):
    df = filter_potential(final_potential, distance_potential, inclination_potential, distance, modal)
    #df = filter_potential(final_potential, distance_potential, age_potential, inclination_potential, modal)
    
    global global_df 
    global_df = df
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
    
    trips_per_zone = calculate_trips_zone_intersection(df, zones, reference)
    
    if density:
        df_total = od_trips if modal == None else od_trips.loc[od_trips['modal'] == modal]
        total_per_zone = calculate_trips_zone_intersection(df_total, zones, reference)
        perc = [0 if total_per_zone[i] == 0 else trips_per_zone[i] / total_per_zone[i] * 100
                for i in range(len(zone_shp))]
        zone_shp['potential_trips'] = perc
        
    else:
        zone_shp['potential_trips'] = trips_per_zone

    # precisa filtrar de novo pra plotar
    zones = zone_shp.loc[zone_shp["NumeroMuni"] == 36] if not plot_rmsp else zone_shp
    
    tooltip_columns = ['NomeZona', 'potential_trips']
    tooltip_aliases = ['District', 'Potential Trips']
    fmap = gr.map_around_sp(the_grid=None,zoom=10,plot_grid=False)
    choro_folium.plot_choropleth(fmap, title, 'YlOrBr', lambda x : x['potential_trips'], 
                          zones, tooltip_columns, tooltip_aliases, plot_rmsp)
    
    if debug:
        #display(df)
        plot_slopes (fmap, df['geometry'], 1000, False)
    fmap.save('maps/' + title + '_potential.html')
    display(fmap)


In [14]:
filter_potential_widgets('Filter trips', plot_potential_trips_per_zone)

interactive(children=(FloatRangeSlider(value=(0.8, 1.0), description='Cycling Potential:\t', layout=Layout(wid…